# Flant Optimization


Alex Wang

Created on: 05/19/2021

# Load Data 

## Load AWS Data (Predictions)

In [1]:
%%time
import pandas as pd
import numpy as np 
import plotly.express as px

from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

TEST=True

prediction_df=pd.read_csv('../input/sample-prediction/Predictions.csv')
prediction_df.columns=[' AvailabilityZone', ' InstanceType', 'Timedel', 'Real_AWS_SpotPrice','Price_Prediction']

region_label=pd.read_csv('../input/flant-processed-data/region_label_dict.csv')
ins_label=pd.read_csv('../input/flant-processed-data/ins_label_dict.csv')
tier=pd.read_csv('../input/flant-processed-data/tier_df.csv')
time_df=pd.read_csv('../input/flant-processed-data/train_val_test_test.csv',usecols=['Timestamp','Timedel']).drop_duplicates()
opt_df=pd.read_csv('../input/flant-processed-data/ins_opt_dict.csv')
add_on=pd.read_csv('../input/flant-aws-price-add-on/SUSE_Linux_price_diff.csv')

prediction_df=ins_label.merge(prediction_df,left_on='InstanceType_label',right_on=' InstanceType').drop(columns=['InstanceType_label',' InstanceType'])
prediction_df=region_label.merge(prediction_df,left_on='AvailabilityZone_label',right_on=' AvailabilityZone').drop(columns=['AvailabilityZone_label',' AvailabilityZone'])
prediction_df=tier.merge(prediction_df,on='InstanceType')
prediction_df=time_df.merge(prediction_df,on='Timedel').drop(columns='Timedel')
prediction_df=opt_df.merge(prediction_df,on='instanceOptimized_label').drop(columns='instanceOptimized_label')

prediction_df['gpu']=np.where(prediction_df['gpu']==-1,0,prediction_df['gpu'])
prediction_df['Timestamp']=pd.to_datetime(prediction_df['Timestamp'].transform(lambda x:x.split('+')[0]))
if TEST:
    np.random.seed(42)
    weights=np.random.random(len(prediction_df))/500
    bi=[int(x<0.93) for x in np.random.random(len(prediction_df))]
    prediction_df['Price_Prediction']=prediction_df['Real_AWS_SpotPrice']*(1+weights*bi)

prediction_df=prediction_df.merge(add_on,on=['InstanceType','AvailabilityZone'])
prediction_df['Price_Prediction']+=prediction_df['SUSE_Linux_price_diff']   # price add on
prediction_df=prediction_df[['Timestamp', 'InstanceType','AvailabilityZone', 'cpu', 'gpu', 'RAM','instanceOptimized', 'Price_Prediction']]

prediction_df

CPU times: user 2.53 s, sys: 351 ms, total: 2.88 s
Wall time: 4.38 s


,Timestamp,InstanceType,AvailabilityZone,cpu,gpu,RAM,instanceOptimized,Price_Prediction
0,2021-04-10 00:00:00,f1.16xlarge,us-east-1a,8,0,976,Acceleration Optimized,4.062966
1,2021-04-10 01:00:00,f1.16xlarge,us-east-1a,8,0,976,Acceleration Optimized,4.062560
2,2021-04-10 02:00:00,f1.16xlarge,us-east-1a,8,0,976,Acceleration Optimized,4.066410
3,2021-04-10 03:00:00,f1.16xlarge,us-east-1a,8,0,976,Acceleration Optimized,4.067093
4,2021-04-10 04:00:00,f1.16xlarge,us-east-1a,8,0,976,Acceleration Optimized,4.067556
...,...,...,...,...,...,...,...,...
304910,2021-04-16 13:00:00,i3en.24xlarge,us-east-1f,96,0,768,Storage Optimized,3.410024
304911,2021-04-16 14:00:00,i3en.24xlarge,us-east-1f,96,0,768,Storage Optimized,3.406494
304912,2021-04-16 15:00:00,i3en.24xlarge,us-east-1f,96,0,768,Storage Optimized,3.404492
304913,2021-04-16 16:00:00,i3en.24xlarge,us-east-1f,96,0,768,Storage Optimized,3.404755


## Load GCP Data

In [2]:
%%time
gcp_raw=pd.read_csv('../input/flant-gcp-data/GCP.csv')
gcp_raw=gcp_raw[gcp_raw.AvailabilityZone.str.startswith('us-')]

gcp_raw['gpu']=0
gcp_raw['Price_Prediction']+=0.2152-0.0902   # price add on
gcp_raw['instanceOptimized']='General Purpose'
gcp_raw=gcp_raw[['InstanceType', 'AvailabilityZone', 'cpu', 'gpu', 'RAM','instanceOptimized', 'Price_Prediction']]

gcp=pd.DataFrame()
for i,line in gcp_raw.iterrows():
    gcp_temp=pd.DataFrame(prediction_df.Timestamp.unique())
    for col in gcp_raw.columns:
        gcp_temp[col]=line[col]
    gcp=pd.concat((gcp,gcp_temp))

gcp.rename({0:'Timestamp'},axis=1,inplace=True)
print(gcp.shape)
gcp.head()

(96936, 8)
CPU times: user 8.35 s, sys: 4.46 ms, total: 8.36 s
Wall time: 8.39 s


,Timestamp,InstanceType,AvailabilityZone,cpu,gpu,RAM,instanceOptimized,Price_Prediction
0,2021-04-10 00:00:00,e2-standard-2,us-central1,2.0,0,8.0,General Purpose,0.145102
1,2021-04-10 01:00:00,e2-standard-2,us-central1,2.0,0,8.0,General Purpose,0.145102
2,2021-04-10 02:00:00,e2-standard-2,us-central1,2.0,0,8.0,General Purpose,0.145102
3,2021-04-10 03:00:00,e2-standard-2,us-central1,2.0,0,8.0,General Purpose,0.145102
4,2021-04-10 04:00:00,e2-standard-2,us-central1,2.0,0,8.0,General Purpose,0.145102


## Load Azure Data 

In [3]:
%%time
azure_raw=pd.read_csv('../input/flant-gcp-data/Azure.csv')
azure_raw=azure_raw[azure_raw.os=='SUSE'].drop(columns=['os','storage'])
azure_raw.columns=['AvailabilityZone','InstanceType', 'cpu', 'RAM', 'Price_Prediction']

azure_raw['gpu']=0
azure_raw['instanceOptimized']='General Purpose'
azure_raw=azure_raw[['InstanceType', 'AvailabilityZone', 'cpu', 'gpu', 'RAM','instanceOptimized', 'Price_Prediction']]

azure=pd.DataFrame()
dates=prediction_df.Timestamp.unique()

for i,line in azure_raw.iterrows():
    azure_temp=pd.DataFrame(dates)
    for col in azure_raw.columns:
        azure_temp[col]=line[col]
    azure=pd.concat((azure,azure_temp))

azure.rename({0:'Timestamp'},axis=1,inplace=True)
print(azure.shape)
azure.head()

(372624, 8)
CPU times: user 59.7 s, sys: 658 ms, total: 1min
Wall time: 1min


,Timestamp,InstanceType,AvailabilityZone,cpu,gpu,RAM,instanceOptimized,Price_Prediction
0,2021-04-10 00:00:00,A0,Central US,1,0,0.75,General Purpose,0.02
1,2021-04-10 01:00:00,A0,Central US,1,0,0.75,General Purpose,0.02
2,2021-04-10 02:00:00,A0,Central US,1,0,0.75,General Purpose,0.02
3,2021-04-10 03:00:00,A0,Central US,1,0,0.75,General Purpose,0.02
4,2021-04-10 04:00:00,A0,Central US,1,0,0.75,General Purpose,0.02


In [4]:
all_price=pd.concat((prediction_df,gcp,azure))
print(all_price.shape)
all_price.head()

(774475, 8)


,Timestamp,InstanceType,AvailabilityZone,cpu,gpu,RAM,instanceOptimized,Price_Prediction
0,2021-04-10 00:00:00,f1.16xlarge,us-east-1a,8.0,0,976,Acceleration Optimized,4.062966
1,2021-04-10 01:00:00,f1.16xlarge,us-east-1a,8.0,0,976,Acceleration Optimized,4.062560
2,2021-04-10 02:00:00,f1.16xlarge,us-east-1a,8.0,0,976,Acceleration Optimized,4.066410
3,2021-04-10 03:00:00,f1.16xlarge,us-east-1a,8.0,0,976,Acceleration Optimized,4.067093
4,2021-04-10 04:00:00,f1.16xlarge,us-east-1a,8.0,0,976,Acceleration Optimized,4.067556


In [5]:
!pip install pulp

     |████████████████████████████████| 40.6 MB 13.1 MB/s 


In [6]:
from pulp import *
def opt_one_hr(filter_df,sample_stamp,Min_CPU_num,Min_GPU_num,Min_RAM_Size,constraints=False):
    sample_df=filter_df[filter_df.Timestamp==sample_stamp]
    sample_df['merge']=sample_df.AvailabilityZone+'_'+sample_df.InstanceType
    sample_df['cpu']=sample_df['cpu'].astype(int)
    sample_df['gpu']=sample_df['gpu'].astype(int)
    sample_df['RAM']=sample_df['RAM'].astype(str).str.replace(',','').astype(float)
    sample_df.sort_values('merge',inplace=True)

    probA=LpProblem("Problem A",LpMinimize)

    #-----------------------------------------------#
    # Define Parameters and parameter dictionaries  #
    #-----------------------------------------------#
    region_ins=sorted(sample_df['merge'].unique())
    cpus=dict(sample_df[['merge','cpu']].values)
    gpus=dict(sample_df[['merge','gpu']].values)
    rams=dict(sample_df[['merge','RAM']].values)
    prices=dict(sample_df[['merge','Price_Prediction']].values)

    #-----------------------------------#
    #      Define decision variables    #
    #-----------------------------------#
    decisions=LpVariable.dicts("x",region_ins,lowBound=0,cat='Integer')

    #-------------------------------#
    #  Define objective function    #
    #-------------------------------#
    costs = lpSum([decisions[i]*prices[i] for i in region_ins])
    probA+=costs

    #-------------------------------#
    #      Define constraints       #
    #-------------------------------#

    probA+=lpSum([decisions[i]*cpus[i] for i in region_ins]) >= Min_CPU_num,"CPU"
    probA+=lpSum([decisions[i]*gpus[i] for i in region_ins]) >= Min_GPU_num,"GPU"
    probA+=lpSum([decisions[i]*rams[i] for i in region_ins]) >= Min_RAM_Size,"RAM"
    
    if constraints:
        for region,ins in constraints:
            merged=region+'_'+ins
            probA+=decisions[merged] <= constraints[(region,ins)]
            
    probA.solve()
    output=[]
    for i in region_ins:
        output.append(decisions[i].varValue)
    return sample_df,output,value(probA.objective)
    
def optimizer(prediction_df,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,REGION=None,opt=None,simple=False,details=False,plot=True,constraints=False,return_json=True):
    filter_df=prediction_df[(prediction_df.Timestamp>=pd.to_datetime(Start_Date)) & (prediction_df.Timestamp<=pd.to_datetime(End_Date))]
    sample_stamps=filter_df.Timestamp.unique()

    if opt:
        if opt not in ['Acceleration','Compute','Memory','Storage']:
            print("Wrong Optimization Type! Supported Optimization Types Are: 'Acceleration','Compute','Memory','Storage'.")
            return None
                
        filter_df=filter_df[filter_df.instanceOptimized.str.contains(opt)]

    if REGION:
        region_dict={
                    'us-east':['us-east-1a', 'us-east-1b', 'us-east-1c', 'us-east-1d','us-east-1e', 
                               'us-east-1f','us-east4', 'us-east1','East US', 'East US 2',],
                    'us-central':['us-central1', 'Central US'],
                    'us-north-central':['North Central US'],
                    'us-south-central':['South Central US'],
                    'us-west-central':['West Central US'],
                    'us-west':['us-west1', 'us-west2','us-west3', 'us-west4','West US', 'West US 2']
                    }

        using=[]
        if type(REGION)==list:
            for region in REGION:
                using.extend(region_dict[region])
        else: using=region_dict[REGION]
        filter_df=filter_df[filter_df.AvailabilityZone.isin(using)]

    simple_res=[]
    # future opt
    for sample_stamp in sample_stamps:
        sample_df,output,cost=opt_one_hr(filter_df,sample_stamp,Min_CPU_num,Min_GPU_num,Min_RAM_Size,constraints)
        ins=sorted(sample_df['merge'].unique())
        
        output_df=pd.DataFrame(output,columns=['Number of Purchase'],index=ins)
        output_df=output_df[output_df['Number of Purchase']>0]
        
        nums=output_df.T.values[0]
        ins=output_df.index
        simple_res.append([sample_stamp,cost]+[str(nums[i])+'_'+str(ins[i]) for i in range(len(nums))])
        
        if simple:return cost

        if details:
            output_df['AvailabilityZone']=list(map(lambda x:x.split('_')[0],output_df.index))
            output_df['InstanceType']=list(map(lambda x:x.split('_')[1],output_df.index))
            output_df.reset_index(inplace=True)
            output_df.columns=['merge','Number of Purchase', 'AvailabilityZone', 'InstanceType']
            output_df=output_df.merge(sample_df[['merge','cpu','gpu','RAM','instanceOptimized','Price_Prediction']],on='merge').drop(columns='merge')
            output_df=output_df[['AvailabilityZone','InstanceType','Number of Purchase','cpu','gpu','RAM','instanceOptimized','Price_Prediction']]
        
            print("Time Stamps: %s\nTotal cost: %.6f\nBest Combination"%(str(sample_stamp)[:19].replace('T',' '),cost))
            display(output_df)
            
    if not details: 
        simple_res=pd.DataFrame(simple_res)
        simple_res.columns=['TimeStamp','Cost']+['InstanceType_%s'%x for x in range(1,simple_res.shape[1]-1)]
        
        inss=['InstanceType_%s'%x for x in range(1,simple_res.shape[1]-1)]
        simple_res['merged']=simple_res[inss].apply(lambda x:'+'.join(x.astype(str)),axis=1)
        if plot:
            gap=simple_res.Cost.max()-simple_res.Cost.min()
            
            fig = px.bar(simple_res, x='TimeStamp', y='Cost',color="merged",text='Cost')
            fig.update_traces(textposition='outside')
            fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
            fig.update_yaxes(ticklabelposition="inside top", title='Best Cost')
            fig.update_yaxes(range=[simple_res.Cost.min()-gap*0.3,simple_res.Cost.max()+gap*0.3], row=1, col=1)
            fig.update_layout(legend=dict(
                                            orientation="h",
                                            yanchor="bottom",
                                            y=1.02,
                                            xanchor="right",
                                            x=1
                                        ))
            fig.show()  
            
#         simple_res=simple_res.groupby('merged',as_index=False).min().sort_values('TimeStamp').drop('merged',axis=1)  
        simple_res['lag']=simple_res['merged'].shift(1)
        simple_res['same']=simple_res['lag']==simple_res['merged']
        simple_res=simple_res[~simple_res['same']].drop(['merged','lag','same'],axis=1)

    if return_json:
        simple_res_arr=simple_res.values
        temp_json={}
        for i in range(len(simple_res_arr)):
            timestamp=str(simple_res_arr[i][0])
            temp_json[timestamp]={}
            temp_json[timestamp]['CombinationCost']=simple_res_arr[i][1]
            temp_json[timestamp]['InstanceComb']={}

            for i,num_region_ins in enumerate(simple_res_arr[i][2:]):
                if type(num_region_ins)==str: 
                    num=int(float(num_region_ins.split('_')[0]))
                    region=num_region_ins.split('_')[1]
                    ins=num_region_ins.split('_')[2]

                    temp_json[timestamp]['InstanceComb']['InstanceType_%s'%(i+1)]={'AvailabilityZone':region,
                                                                                   'InstanceType':ins,
                                                                                   'Number':num}      
        return temp_json
    else:
        return simple_res

In [7]:
%%time
# No constraint
Min_CPU_num=0
Min_GPU_num=0
Min_RAM_Size=1000
Start_Date='2021-04-11 6:00:00'
End_Date='2021-04-11 18'

res=optimizer(all_price,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,return_json=False)
# res=optimizer(prediction_df,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,details=False,return_json=False)
# optimizer(prediction_df,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,details=True)
# optimizer(prediction_df,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date)
# optimizer(gcp,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date)
# optimizer(azure,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date)
# res=optimizer(prediction_df,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,plot=True)
res

CPU times: user 5.76 s, sys: 446 ms, total: 6.2 s
Wall time: 7.83 s


,TimeStamp,Cost,InstanceType_1,InstanceType_2
0,2021-04-11 06:00:00,1.553600,39.0_us-east-1a_t4g.micro,1.0_us-west1_m1-ultramem-40
1,2021-04-11 07:00:00,1.553601,39.0_us-east-1f_t4g.micro,1.0_us-west1_m1-ultramem-40
2,2021-04-11 08:00:00,1.553619,39.0_us-east-1c_t4g.micro,1.0_us-west1_m1-ultramem-40
3,2021-04-11 09:00:00,1.553600,39.0_us-east-1a_t4g.micro,1.0_us-west1_m1-ultramem-40
4,2021-04-11 10:00:00,1.553600,39.0_us-east-1f_t4g.micro,1.0_us-west1_m1-ultramem-40
5,2021-04-11 11:00:00,1.553600,39.0_us-east-1c_t4g.micro,1.0_us-west1_m1-ultramem-40
8,2021-04-11 14:00:00,1.553603,39.0_us-east-1f_t4g.micro,1.0_us-west1_m1-ultramem-40
10,2021-04-11 16:00:00,1.553600,39.0_us-east-1c_t4g.micro,1.0_us-west1_m1-ultramem-40
11,2021-04-11 17:00:00,1.553688,39.0_us-east-1d_t4g.micro,1.0_us-west1_m1-ultramem-40
12,2021-04-11 18:00:00,1.553600,39.0_us-east-1c_t4g.micro,1.0_us-west1_m1-ultramem-40


In [8]:
# Add constraints
constraints={
    ('us-east-1a','t4g.micro'):5
            }

res=optimizer(all_price,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,constraints=constraints,return_json=True)
pprint(res)

{'2021-04-11 06:00:00': {'CombinationCost': 1.5536025365509634,
                         'InstanceComb': {'InstanceType_1': {'AvailabilityZone': 'us-east-1a',
                                                             'InstanceType': 't4g.micro',
                                                             'Number': 5},
                                          'InstanceType_2': {'AvailabilityZone': 'us-east-1b',
                                                             'InstanceType': 't4g.micro',
                                                             'Number': 34},
                                          'InstanceType_3': {'AvailabilityZone': 'us-west1',
                                                             'InstanceType': 'm1-ultramem-40',
                                                             'Number': 1}}},
 '2021-04-11 07:00:00': {'CombinationCost': 1.5536006503626598,
                         'InstanceComb': {'InstanceType_1': {'AvailabilityZone': 'us-e

In [9]:
# Add more constraints & region
REGION='us-east'
constraints={
    ('us-east-1a','t4g.micro'):5,
    ('us-east-1b','t4g.micro'):5,
            }

res=optimizer(all_price,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,REGION,constraints=constraints,return_json=True)
pprint(res)

{'2021-04-11 06:00:00': {'CombinationCost': 1.5536125294034888,
                         'InstanceComb': {'InstanceType_1': {'AvailabilityZone': 'us-east-1a',
                                                             'InstanceType': 't4g.micro',
                                                             'Number': 5},
                                          'InstanceType_2': {'AvailabilityZone': 'us-east-1b',
                                                             'InstanceType': 't4g.micro',
                                                             'Number': 5},
                                          'InstanceType_3': {'AvailabilityZone': 'us-east-1d',
                                                             'InstanceType': 't4g.micro',
                                                             'Number': 29},
                                          'InstanceType_4': {'AvailabilityZone': 'us-east1',
                                                             '

In [10]:
# ind=0


# change_df=prediction_df.copy()
# change_df['lag']=change_df.groupby(['InstanceType','AvailabilityZone']).Price_Prediction.shift(1)
# change_df['diff']=change_df.Price_Prediction-change_df['lag']
# change_df=change_df[change_df['diff']!=0]
# change_df['merge']=change_df['AvailabilityZone']+'_'+change_df['InstanceType']
# # change_df[change_df['merge']==inss[ind]]
# change_df[change_df['merge']==merges[ind]]

In [11]:
# change_df=prediction_df.groupby(['AvailabilityZone','InstanceType'],as_index=False)
# change_df=change_df.Price_Prediction.nunique()
# change_df=change_df[change_df.Price_Prediction>2]
# change_df['merge']=change_df['AvailabilityZone']+'_'+change_df['InstanceType']
# changes=change_df['merge'].values
# change_df

In [12]:
# count=1
# flag=0
# cob=[]
# for cpu in range(1,150):
#     for gpu in range(0,10):
#         for ram in range(1,2000):
#             cob.append([cpu,gpu,ram])
# np.random.shuffle(cob)

# for cpb, gpu,ram in cob:
#     Min_CPU_num=cpu
#     Min_GPU_num=gpu
#     Min_RAM_Size=ram
#     Start_Date='2021-04-10 23:00:00'
#     End_Date=Start_Date
        
#     res=optimizer(prediction_df,Min_CPU_num,Min_GPU_num,Min_RAM_Size,Start_Date,End_Date,plot=False)
#     merges=['_'.join(x.split('_')[1:]) for x in res.values[0][2:]]
    
#     for x in merges:
#         if x in changes:
#             flag =1 
#     if flag: break
#     if not count%10:print(count)
#     count+=1

# print('done!')
# print(Min_CPU_num,Min_GPU_num,Min_RAM_Size)